# DriveSight Data Querying

**Purpose:** This notebook contains practical SQL queries for each key use cases.

Use Case 1: Active Customer Type Share per Car Brand (Monthly)

Model: `gld_active_customer_type_share`

Use Case 2: Customer Type Distribution per Terms

Model: `gld_customer_type_distribution_per_term`

Use Case 3: Daily Fleet Volume (Infleeted/Defleeted Cars)

Model: `gld_cars_infleet_defleet_volume`

Use Case 4: City with Greatest Weekly Delivery Increase

Model: `gld_city_weekly_delivery_increase`


In [ ]:
# Installing required packages
# !pip install numpy
# !pip install pandas
# !pip install duckdb

In [3]:
# Import required libraries for data analysis
import duckdb
import numpy
import pandas

In [4]:
# Connect to the DuckDB database file
# NOTE: Path is relative to notebook location
dev_dbdir = '../../FINN.duckdb'

In [5]:
def fetch_df(query):
    conn = duckdb.connect('../../FINN.duckdb')
    try:
        result = conn.sql(query).fetchdf()
        conn.close()
    except:
        conn.close()
    return result

In [6]:
## 📊 Customer Acquisition Use Cases

### Use Case 1: Active Customer Type Share per Car Brand (Monthly)
# Customer type share by brand and month
query = """
SELECT 
    month,
    brand,
    customer_type,
    active_customers_per_type_car_brand_month,
    active_customers_per_car_brand_month,
    customer_type_share,
    num_cars_used,
    avg_term_months
FROM main_gold.gld_active_customer_type_share
ORDER BY month DESC, brand, customer_type;
"""
act_cust_df = fetch_df(query)
act_cust_df.head()

,month,brand,customer_type,active_customers_per_type_car_brand_month,active_customers_per_car_brand_month,customer_type_share,num_cars_used,avg_term_months


In [13]:
# Advanced analysis: B2B vs B2C comparison
query = """
SELECT 
    month,
    brand,
    SUM(CASE WHEN customer_type = 'b2b' THEN customer_type_share ELSE 0 END) AS b2b_share,
    SUM(CASE WHEN customer_type = 'b2c' THEN customer_type_share ELSE 0 END) AS b2c_share,
    SUM(CASE WHEN customer_type = 'other' THEN customer_type_share ELSE 0 END) AS other_share
FROM main_gold.gld_active_customer_type_share
GROUP BY month, brand
ORDER BY month DESC, brand;
"""
b2b_vs_b2c_df = fetch_df(query)
b2b_vs_b2c_df.head()

,month,brand,b2b_share,b2c_share,other_share


In [16]:
## 📊 Customer Acquisition Use Cases

### Use Case 2: Customer Type Distribution per Terms
# Customer distribution by term length
query = """
SELECT 
    term_months,
    customer_type,
    customers_per_type_term,
    subscriptions_per_type_term,
    customers_per_term,
    customer_type_share,
    avg_monthly_rate_per_type_term,
    total_revenue_per_type_term
FROM main_gold.gld_customer_type_distribution_per_term
ORDER BY term_months, customer_type;
"""
cust_dist_df = fetch_df(query)
cust_dist_df.head()

,term_months,customer_type,customers_per_type_term,subscriptions_per_type_term,customers_per_term,customer_type_share,avg_monthly_rate_per_type_term,total_revenue_per_type_term
0,6,b2b,10,10,108.0,0.0926,533.000000,31980.0
1,6,b2c,98,104,108.0,0.9074,523.653846,326760.0
2,12,b2b,50,98,545.0,0.0917,567.959184,667920.0
3,12,b2c,495,988,545.0,0.9083,557.236842,6606600.0


In [15]:
# Revenue analysis: Revenue contribution by customer type and term
query = """
SELECT 
    term_months,
    SUM(CASE WHEN customer_type = 'b2b' THEN total_revenue_per_type_term ELSE 0 END) AS b2b_revenue,
    SUM(CASE WHEN customer_type = 'b2c' THEN total_revenue_per_type_term ELSE 0 END) AS b2c_revenue,
    SUM(total_revenue_per_type_term) AS total_revenue
FROM main_gold.gld_customer_type_distribution_per_term
GROUP BY term_months
ORDER BY term_months;
"""
revenue_df = fetch_df(query)
revenue_df.head()

,term_months,b2b_revenue,b2c_revenue,total_revenue
0,6,31980.0,326760.0,358740.0
1,12,667920.0,6606600.0,7274520.0


In [9]:
## 🚛 Operations Use Cases

### Use Case 3: Daily Fleet Volume (Infleeted/Defleeted Cars)
# Basic query: Daily fleet changes
query = """
SELECT 
    date,
    infleeted_count,
    defleeted_count,
    cumulative_fleet_size,
    net_fleet_change
FROM main_gold.gld_cars_infleet_defleet_volume
WHERE infleeted_count != 0 OR defleeted_count != 0
ORDER BY date DESC
LIMIT 30;
"""
daily_fleet_df = fetch_df(query)
daily_fleet_df.head()

,date,infleeted_count,defleeted_count,cumulative_fleet_size,net_fleet_change
0,2024-02-22,0,5,598,-5
1,2024-02-21,0,2,603,-2
2,2024-02-20,0,3,605,-3
3,2024-02-19,0,2,608,-2
4,2024-02-18,0,3,610,-3


In [13]:
# Weekly aggregation: Fleet changes by week
query = """
SELECT 
    DATE_TRUNC('week', date) AS week,
    SUM(infleeted_count) AS weekly_infleeted,
    SUM(defleeted_count) AS weekly_defleeted,
    SUM(net_fleet_change) AS weekly_net_change,
FROM main_gold.gld_cars_infleet_defleet_volume
GROUP BY DATE_TRUNC('week', date)
HAVING SUM(net_fleet_change) > 0
ORDER BY week DESC;
"""
weekly_fleet_df = fetch_df(query)
weekly_fleet_df.head()

,week,weekly_infleeted,weekly_defleeted,weekly_net_change
0,2023-12-11,10.0,7.0,3.0
1,2023-12-04,14.0,13.0,1.0
2,2023-11-27,12.0,9.0,3.0
3,2023-10-30,19.0,12.0,7.0
4,2023-10-16,14.0,13.0,1.0


In [10]:
## 🚛 Operations Use Cases

### Use Case 4: City with Greatest Weekly Delivery Increase
# Basic query: Cities with highest weekly delivery growth
query = """
SELECT
    delivery_week,
    city,
    current_week_deliveries,
    previous_week_deliveries,
    deliveries_diff,
    growth_rate
FROM main_gold.gld_city_weekly_delivery_increase
WHERE deliveries_diff > 0
ORDER BY delivery_week DESC, deliveries_diff DESC;
"""
city_delivery_df = fetch_df(query)
city_delivery_df.head()

,delivery_week,city,current_week_deliveries,previous_week_deliveries,deliveries_diff,growth_rate
0,2023-11-20,Vechta,2,1,1,100.0
1,2023-09-11,Eisenach,2,1,1,100.0
2,2023-07-31,Havelberg,2,1,1,100.0
3,2023-07-24,Schwabmünchen,2,1,1,100.0
4,2023-07-03,Starnberg,2,1,1,100.0


In [18]:
# Top performing cities: Highest growth cities by week
query = """
SELECT
    delivery_week,
    city,
    deliveries_diff,
    current_week_deliveries
FROM main_gold.gld_city_weekly_delivery_increase
WHERE deliveries_diff > 0  -- Only positive growth
    AND delivery_rank <= 5  -- Top 5 cities
ORDER BY delivery_week DESC, delivery_rank;
"""
top_cities_df = fetch_df(query)
top_cities_df.head()

,delivery_week,city,deliveries_diff,current_week_deliveries
0,2023-11-20,Vechta,1,2
1,2023-09-11,Eisenach,1,2
2,2023-07-31,Havelberg,1,2
3,2023-07-24,Schwabmünchen,1,2
4,2023-07-03,Starnberg,1,2


In [17]:
%reset -f  # Clear all variables to free memory

Don't know how to reset  #, please run `%reset?` for details
Don't know how to reset  clear, please run `%reset?` for details
Don't know how to reset  all, please run `%reset?` for details
Don't know how to reset  variables, please run `%reset?` for details
Don't know how to reset  to, please run `%reset?` for details
Don't know how to reset  free, please run `%reset?` for details
Don't know how to reset  memory, please run `%reset?` for details
